In [1]:
using LinearAlgebra
using SparseArrays
using Distributions
using PyCall

@pyimport numpy 

function gmres(A, B, X0, max_iter)
    r = B .- A * X0
    x = Array{Float64, 2}[]
    q = Array{Any}(zeros(max_iter))
    #push!(x, r)
    q[1] = r/norm(r)
    h = zeros((max_iter + 1, max_iter))
    
    for m=1:max_iter
        y = A * q[m]
        
        for j=1:m-1
            h[j, m] = (q[j]' * y)[1]
            y = y - h[j, m] * q[j] 
        end
        
        h[m + 1, m] = norm(y)
        
        if (h[m + 1, m] != 0) && (m != max_iter)
            q[m + 1] = y / h[m + 1, m]
        end

        b = zeros(max_iter + 1)
        b[1] = norm(r)
        result = (h \ b)

        if m > max_iter - 2
            z = reduce(hcat, q[:])
        else
            z = q
        end

        append!(x, [numpy.dot(z, result)+X0])
    end
    
    return x 
end


gmres (generic function with 1 method)

In [2]:
A = Symmetric(sprand(20,20,0.5))
B = rand(Uniform(0,1), 20, 1)
X0 = rand(Uniform(0,1), 20, 1)
max_iter = 1000
x = gmres(A, B, X0, max_iter)
x[max_iter]

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


20×1 Matrix{Float64}:
 -0.2148566451867524
  0.4763420382091518
 -0.24706307023609997
 -0.29802171734052085
  1.268744193039384
 -0.1751501232005982
 -0.020171428368478767
  0.031726529445988305
  0.4382589436215707
  0.7104935194364352
 -0.13807859927913185
  0.2971068522997585
  0.23771295717614582
  0.4444602751966018
  1.3488285301349199
 -0.8662739035478776
  0.4454717996329342
 -0.5529992101992618
  0.23360028990629322
  0.11280668121168735

In [3]:
A\B

20×1 Matrix{Float64}:
 -0.21485664518850192
  0.4763420382065191
 -0.24706307023404106
 -0.2980217173394895
  1.268744193040703
 -0.17515012319978135
 -0.020171428372504963
  0.03172652944453358
  0.43825894361804907
  0.7104935194387935
 -0.13807859927718447
  0.2971068523053185
  0.23771295717306612
  0.44446027519517906
  1.3488285301373535
 -0.866273903548658
  0.445471799635013
 -0.5529992101998369
  0.23360028990701626
  0.112806681209323